# Dog Breed CNN Classification with TensorFlow

## Импорт библиотек

In [ ]:
import shutil
import zipfile
import os

from google.colab import drive
from PIL import Image
import tensorflow as tf
import numpy as np
from tensorflow import keras
from matplotlib import pyplot as plt
import pandas as pd
import sklearn as skl

tf.test.gpu_device_name()

## Подключение GoogleDrive (Для Google Colab)

In [ ]:
drive.mount('/content/drive')

## Загрузка набора данных из GoogleDrive

In [ ]:
zip_file_path = '/content/drive/MyDrive/Datasets/DogBreeds.zip'
shutil.copy(zip_file_path, '/content/')

unzip_dir = '/content/DogBreeds/'
with zipfile.ZipFile('/content/DogBreeds.zip', 'r') as zip_ref:
    zip_ref.extractall(unzip_dir)

print(os.listdir(unzip_dir))

## Предпросмотр данных

Просмотр csv-файла

In [ ]:
df = pd.read_csv('/content/DogBreeds/labels.csv')
df['filepath'] = '/content/DogBreeds/train/' + df['id'] + '.jpg'
print(df.head())
print(f'В наборе данных {df['breed'].nunique()} классов')

Оценка количества изображений

In [ ]:
plt.figure(figsize=(12, 7))
df['breed'].value_counts().plot.bar()
plt.title('Число изображений для каждого класса')
plt.xlabel('Класс')
plt.ylabel('Число')
plt.axis('off')
plt.show()

Предпросмотр случайных изображений

In [ ]:
plt.figure(figsize=(16, 10))
for i in range(9):
    plt.subplot(3, 3, i + 1)
    k = np.random.randint(0, len(df))
    img = Image.Image(df.loc[k, 'filepath'])
    plt.imshow(img)
    plt.title(df.loc[k, 'breed'])
    plt.axis('off')
    
plt.show()

## Загрузка данных

In [ ]:
# Кодирование пород числом
LE = skl.preprocessing.LabelEncoder()
df['breed'] = LE.fit_transform(df['breed'])

features = df['filepath'] 
target = df['breed'] 
  
x_train, x_val, y_train, y_val = skl.model_selection.train_test_split(features, target, test_size=0.15, random_state=10)


def decode_image(filepath, label=None): 
    image = tf.io.read_file(filepath) 
    image = tf.image.decode_jpeg(img, channels=3)  # Ensure 3 channels for color images
    image = tf.image.resize(image, 128) 
    image = tf.cast(image, tf.float32) / 255.0 # the pixels will be between [0, 1].
    
    if label is None: 
        return image
  
    label = tf.one_hot(label, depth=120, dtype=tf.float32)
    return image, label


train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .map(decode_image)
    .batch(32)
)

validation_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_val, y_val))
    .map(decode_image)
    .batch(32)
)

for image, label in train_dataset.take(1): 
    print(f'Форма пакета изображений: {image.shape}, Форма пакета меток: {label.shape}')

## Построение модели

In [ ]:
inception_v3 = keras.applications.InceptionV3(
    input_shape=(128, 128, 3),
    weights = 'imagenet',
    include_top = False
)

for layer in inception_v3.layers:
    layer.trainable = False

data_augmentation = keras.Sequential([
    keras.layers.RandomFlip('horizontal'),
    keras.layers.RandomRotation(0.1),
    keras.layers.RandomZoom(0.2)
])

last_layer = inception_v3.get_layer('mixed7')
print(f'Вывод последнего слоя: {last_layer.output_shape}')
last_output = last_layer.output

inputs = keras.Input(shape=(128, 128, 3))
x = inception_v3(inputs)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(256, activation = 'relu')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dense(256, activation = 'relu')(x)
x = keras.layers.Dropout(0.3)(x)
x = keras.layers.BatchNormalization()(x)
outputs = keras.layers.Dense(120, activation = 'softmax')(x)

model = keras.Model(inputs, outputs)

## Компиляция модели

In [ ]:
model.compile(
    optimizer = 'adam',
    loss = keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics = ['AUC']
)

## Создание обратных вызовов

In [ ]:
def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1 **(epoch / s)
    return exponential_decay_fn


exp_decay_fn = exponential_decay(0.01, 20)

callbacks = [
    tf.keras.callbacks.LearningRateScheduler(exp_decay_fn),
    tf.keras.callbacks.ModelCheckpoint('dogbreeds.keras', save_best_only=True),
    tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
]

## Обучение модели

In [ ]:
history = model.fit(
    train_dataset, 
    validation_data=validation_dataset, 
    epochs=50, 
    verbose=1, 
    callbacks=callbacks
)

## Визуализация обучения

In [ ]:
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, 'bo', label='Точность на этапе обучения')
plt.plot(epochs, val_accuracy, 'b', label='Точность на этапе проверки')
plt.title('Точность на этапах обучения и проверки')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'ro', label='Потери на этапе обучения')
plt.plot(epochs, val_loss, 'r', label='Потери на этапе проверки')
plt.title('Потери на этапах обучения и проверки')
plt.legend()
plt.show()